In [1]:
# import all the necessary libraries

from main import *
import pandas as pd
import re

from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.stem import SnowballStemmer
from bs4 import BeautifulSoup
import string

# import all the necessary libraries
from gensim.models import Word2Vec

C:\Users\oguzk\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [36]:
# load all the translation files

csen = pd.read_csv("corpus\cs-en\scores.csv")


In [48]:
# Preprocessing

# Based on the target language a different preprocesing method is followed

# English
    #Czech
csen = csen.drop(["source", "annotators"], axis=1)
csen["pt"] = not_that_clean(csen["translation"], False, False)
csen["ref"] = not_that_clean(csen["reference"], False, False)


# Finnish

# Chinese


In [52]:
csen.head()

,reference,translation,z-score,avg-score,pt,ref
0,You will then grab the weapon between your for...,You then grasp the gun between your forearm an...,-0.675383,60.000000,"[you, then, grasp, the, gun, between, your, fo...","[you, will, then, grab, the, weapon, between, ..."
1,"But if New York is changed, then it's also a r...","But if New York is change, it is also reinvent...",-0.829403,44.000000,"[but, if, new, york, is, change, it, is, also,...","[but, if, new, york, is, changed, then, it, s,..."
2,I have been thinking over and over again over ...,I have thought long and hard over the course o...,0.803185,96.500000,"[i, have, thought, long, and, hard, over, the,...","[i, have, been, thinking, over, and, over, aga..."
3,They find another way to cheat somewhere.,They will find another way how to defraud others.,0.563149,90.500000,"[they, will, find, another, way, how, to, defr...","[they, find, another, way, to, cheat, somewhere]"
4,The report on the replacement of the president...,The news of the replacement at the top of the ...,0.021549,74.666667,"[the, news, of, the, replacement, at, the, top...","[the, report, on, the, replacement, of, the, p..."


In [51]:
import numpy as np

def ngram_calc(pt, gt, mode="precision", ngram=2):
    ptgram = [" ".join(pt[i:i+ngram]) for i in range(len(pt[:len(pt)-ngram+1]))]
    gtgram = [" ".join(gt[i:i+ngram]) for i in range(len(gt[:len(gt)-ngram+1]))]
    if mode == "precision":
        den = len(ptgram)
        nom = 0
        for j in ptgram:
            if j in gtgram:
                nom += 1
                gtgram.remove(j)
        try:
            tot = nom/den
        except ZeroDivisionError:
             tot = 0
        return tot
    elif mode == "recall":
        den = len(gtgram)
        nom = 0
        for j in gtgram:
            if j in ptgram:
                nom += 1
                ptgram.remove(j)
        try:
            tot = nom/den
        except ZeroDivisionError:
             tot = 0
        return tot

In [34]:
def not_that_clean(text_list, lemmatize, stemmer):
    """
    Function that a receives a list of strings and preprocesses it.
    
    :param text_list: List of strings.
    :param lemmatize: Tag to apply lemmatization if True.
    :param stemmer: Tag to apply the stemmer if True.
    """
    stop = set(stopwords.words('english'))
    lemma = WordNetLemmatizer()
    snowball_stemmer = SnowballStemmer('english')
    
    updates = []
    for j in range(len(text_list)):
        
        text = text_list[j]

        text = text.lower()
        text = re.sub("[^a-zA-Z]", ' ', text)

        text = word_tokenize(text)

        # REMOVE STOP WORDS
        #text = [word for word in text if word not in stop]

        #LEMMATIZATION
        if lemmatize:
            text = [lemma.lemmatize(word) for word in text]
        
        #STEMMER
        if stemmer:
            text = [snowball_stemmer.stem(word) for word in text.split()]
        
        updates.append(text)
        
    return updates

In [ ]:
def ngram_calc(pt, gt, mode="precision", ngram=2):
df['col_3'] = df.apply(lambda x: f(x.col_1, x.col_2), axis=1)

In [29]:
csen["2gram"] = csen.apply(lambda x: ngram_calc(x.pt, x.ref, ngram=2), axis=1)

In [32]:
csen["2gram"].isna().sum()

5

In [40]:
csen.pt.apply(lambda x: len(x)).value_counts()

12    557
15    551
11    518
16    504
17    491
     ... 
56      4
76      4
46      4
62      4
71      4
Name: pt, Length: 62, dtype: int64

In [41]:
csen[csen['pt'].map(len) <= 3]

,source,reference,translation,z-score,avg-score,annotators,pt,ref
22,To je k ničemu.,That's useless.,That is useless.,0.606375,97.000000,2,"[that, is, useless]","[that, s, useless]"
28,Zapomeň na vzrušení.,Forget the excitement.,Forget the hype.,0.111094,84.000000,2,"[forget, the, hype]","[forget, the, excitement]"
51,Tolerantní město.,Tolerant city.,A tolerant city.,0.907353,95.666667,3,"[a, tolerant, city]","[tolerant, city]"
220,Hermannová: Byla zvláštní.,Hermann: It was strange.,It was strange.,0.662727,84.000000,2,"[it, was, strange]","[hermann, it, was, strange]"
371,Podobný případ.,A similar case.,A similar case.,1.062107,100.000000,1,"[a, similar, case]","[a, similar, case]"
...,...,...,...,...,...,...,...,...
11161,Ukrajinská armáda.,Ukrainian army.,The Ukrainian army.,0.797377,88.000000,1,"[the, ukrainian, army]","[ukrainian, army]"
11251,Banánové listy.,Banana leaves.,Banana leaves.,1.330578,100.000000,1,"[banana, leaves]","[banana, leaves]"
11313,Zlato číslo 22.,Gold number 22.,Gold medal No 22,-1.982803,26.000000,1,"[gold, medal, no]","[gold, number]"
11371,Spíše naopak.,rather the opposite.,Quite the contrary.,1.123697,100.000000,2,"[quite, the, contrary]","[rather, the, opposite]"


In [34]:
df['Length'] = df['CreationDate'].str.len()

['The',
 'monument',
 'dignified',
 'piecemeal',
 'place',
 'stands',
 'history',
 'practically',
 'immediate',
 'vicinity',
 'tragedy',
 'whoever',
 'wants',
 'may',
 'bow',
 'memory',
 'sufferers',
 'murdered']